# PDAML Report: Parameter Estimation
This checkpoint is based around a particle decay from which you will measure a parameter related
to the matter/anti-matter asymmetry of the Universe.
This relevant decay X → D has the following PDF:
P(t; V, τ, ∆ms) ∝ (1 + V sin(∆mt)) × exp(−t/τ )
where
- t is the observable quantity - the decay time of each decay;
- τ is a lifetime parameter;
- V is a parameter which measures matter/anti-matter asymmetry and has the value zero if
the universe is symmetric (which we know it isnt !);
- ∆m is a mass difference parameter which leads to sinusoidal oscillations superimposed on
the exponential decay.
The nominal values of the parameters are
- τ = 1.5
- V = 0.1
- ∆m = 20.0

In [1]:
# preamble
%matplotlib inline
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.stats import norm
import scipy.integrate as integrate

from iminuit import Minuit
from iminuit.cost import UnbinnedNLL
from iminuit.cost import BinnedNLL

In [2]:
# pdf as a function
def pdf(t, v, tau, m, t_min, t_max):
    # get the scaling using scipy integrate
    scaling = integrate.quad(lambda t: (1 + v * np.sin(m * t)) * np.exp(- t / tau), t_min, t_max)[0]
    return (1 + v * np.sin(m * t)) * np.exp(- t / tau) / scaling

In [3]:
# check if the pdf is normalised, should be very close to 1
print("integral of the pdf over the whole range:",integrate.quad(lambda x: pdf(x, 0.1, 1.5, 20, 0, 10), 0, 10)[0])

integral of the pdf over the whole range: 1.0


In [4]:
# cdf of the pdf
def cdf(ts, v, tau, m, t_min, t_max):
    return np.cumsum(pdf(ts, v, tau, m, t_min, t_max))

## Part 1: Estimating statistical precision [4 marks]
Use the method of pseudo-experiments (toy Monte Carlo) to determine the expected statistical
precision with which one could measure each of the parameters with 
(i) 10,000 events and 
(ii) 100,000 events.
Assume perfect detector-resolution/perfect time measurements.

In [5]:
# define monte carlo method
def box_method(tau, v, m, runs, t_min, t_max):
    f_max = 5
    # using the box method to generate values from the pdf
    ## generate random numbers uniformly [t_min, t_max]
    ts = []
    while len(ts) < runs:
        x = np.random.uniform(t_min, t_max, runs)
        ## compute the pdf for these
        y1_s = pdf(x, v, tau, m, t_min, t_max)
        ## generate a second set of random numbers uniformly [0, f_max]
        y2_s = np.random.uniform(0, f_max, runs)
        x = x[y2_s < y1_s]
        ts.extend(x)
    if len(ts) > runs:
        return ts[:runs]
    return ts



In [6]:
# perform a fit for 10_000 runs of the monte carlo method
## create data
tau = 1.5
v = 0.1
m = 20
t_min = 0
t_max = 20

In [7]:
def parameter_fitting(runs):
    data = box_method(tau, v, m, runs, t_min, t_max)
    binned_data, bins = np.histogram(data, bins=100, density=True)
    bin_width = np.diff(bins)[0]
    # cost = UnbinnedNLL(data, pdf*bin_width)
    cost = BinnedNLL(binned_data, bins, cdf)
    minuit = Minuit(cost, tau=1.5, v=0.1, m=20, t_min=t_min, t_max=t_max)
    minuit.errordef=0.5
    minuit.fixed['t_min'] = True
    minuit.fixed['t_max'] = True
    minuit.migrad()
    minuit.hesse()
    # display the estimations and the errors
    for p, val, e in zip(minuit.parameters, minuit.values, minuit.errors):
        # round to 2 sig fig
        val = np.format_float_positional(val, precision=2, unique=False, fractional=False, trim='k')
        e = np.format_float_positional(e, precision=2, unique=False, fractional=False, trim='k')
        print(f"{p} = {val} +/- {e}")
    # store tau and error for later
    tau_idx = minuit.parameters.index('tau')
    return minuit.values[tau_idx], minuit.errors[tau_idx]


In [8]:
print("Parameters and errors for 10,000 events:")
unbiased_tau, tau_error = parameter_fitting(10_000)
print("Parameter fitting and eror for 100,000 events:")
parameter_fitting(100_000)
pass

Parameters and errors for 10,000 events:
v = 0.0048 +/- 0.39
tau = 1.6 +/- 0.38
m = 19. +/- 9.7
t_min = 0.0 +/- 0.10
t_max = 20. +/- 0.20
Parameter fitting and eror for 100,000 events:


C:\Users\Gamer2021\anaconda3\envs\daml\lib\site-packages\ipykernel_launcher.py:4: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  after removing the cwd from sys.path.
C:\Users\Gamer2021\anaconda3\envs\daml\lib\site-packages\iminuit\cost.py:30: RuntimeWarning: invalid value encountered in log
  return n * (np.log(n + 1e-323) - np.log(mu + 1e-323))
C:\Users\Gamer2021\anaconda3\envs\daml\lib\site-packages\ipykernel_launcher.py:4: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underes

v = -0.31 +/- 0.59
tau = 1.7 +/- 0.54
m = 36. +/- 0.63
t_min = 0.0 +/- 0.10
t_max = 20. +/- 0.20


## Part 2: Estimating possible bias due to time resolution [3 marks]
In reality the decay time is measured with a resolution (random error) with a standard deviation
of σ = fτ where f is some fraction. What this means is that if the true decay time is t_true,
then it is actually measured as t where t is distributed around t_true with a Gaussian probability
distribution with standard deviation σ.

Determine the bias which will be introduced to the measurement of each of the parameters, when
the data is subject to the resolution effect, but this is not included into the PDF used for fitting
(measuring) the parameters.

You should do this for the case of a 10,000 event data sample. Use both f = 0.01 and f = 0.03 and
in each case compare the bias (if any) to the expected statistical precision.

*Hint: this means producing monte-carlo data where you include the effect of resolution on the decay times
you create randomly, but then fitting to it without allowing for this in the PDF*

In [9]:
# modify the monte carlo method for tau with random error
def biased_box_method(tau, v, m, runs, t_min, t_max, f):
    f_max = 5
    tau = norm.pdf(f * tau)
    # using the box method to generate values from the pdf
    ## generate random numbers uniformly [t_min, t_max]
    ts = []
    while len(ts) < runs:
        x = np.random.uniform(t_min, t_max, runs)
        ## compute the pdf for these
        y1_s = pdf(x, v, tau, m, t_min, t_max)
        ## generate a second set of random numbers uniformly [0, f_max]
        y2_s = np.random.uniform(0, f_max, runs)
        x = x[y2_s < y1_s]
        ts.extend(x)
    if len(ts) > runs:
        return ts[:runs]
    return ts

In [10]:
def biased_parameter_fitting(runs, f):
    data = biased_box_method(tau, v, m, 10_000, t_min, t_max, f)
    binned_data, bins = np.histogram(data, bins=100, density=True)
    bin_width = np.diff(bins)[0]
    # cost = UnbinnedNLL(data, pdf*bin_width)
    cost = BinnedNLL(binned_data, bins, cdf)
    minuit = Minuit(cost, tau=1.5, v=0.1, m=20, t_min=t_min, t_max=t_max)
    minuit.errordef=0.5
    minuit.fixed['t_min'] = True
    minuit.fixed['t_max'] = True
    minuit.migrad()
    minuit.hesse()
    # return tau
    tau_idx = minuit.parameters.index('tau')
    return minuit.values[tau_idx]

In [11]:
# perform a fit for 10_000 runs of the monte carlo method
## create data
tau = 1.5
v = 0.1
m = 20
t_min = 0
t_max = 20

print("Bias for f=0.01")
print(np.abs(unbiased_tau - biased_parameter_fitting(10_000, 0.01)))
print("Bias for f=0.03")
print(np.abs(unbiased_tau - biased_parameter_fitting(10_000, 0.03)))
print("tau error:", tau_error)


Bias for f=0.01


C:\Users\Gamer2021\anaconda3\envs\daml\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
C:\Users\Gamer2021\anaconda3\envs\daml\lib\site-packages\ipykernel_launcher.py:4: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  after removing the cwd from sys.path.


1.12822005086006
Bias for f=0.03
1.132088122893335
tau error: 0.37616042978925407


## Part 3: Estimating a systematic error due to time acceptance [3 marks]
The method of measuring decay-time (i.e. in some detector) is thought to exhibit a decay-time
acceptance given by

a(t) = (1 + st)

where s is only known with a precision of s = 0 ± 0.03

Determine a suitable systematic error to assign to the measurement of each of the parameters
due to this this limited knowledge of a(t) and in each case compare the this systematic error (if
any) to the expected statistical precision.

*Hint: This is a stretch part of the project. You will need to understand all of the slides on acceptance in
the notes to do this. For this part you will have to generate data and fit it twice. This means you will have
to include the acceptance function in the PDF and in its normalisation.*


In [12]:
# modify the pdf to include the acceptance
def acceptance_pdf(t, v, tau, m, s, t_min, t_max):
    # get the scaling using scipy integrate
    scaling = integrate.quad(lambda t: (1 + v * np.sin(m * t)) * np.exp(- t / tau) * (1 + s*t), t_min, t_max)[0]
    return (1 + v * np.sin(m * t)) * np.exp(- t / tau) * (1 + s * t) / scaling

In [13]:
def acceptance_cdf(t, v, tau, m, s, t_min, t_max):
    return np.cumsum(acceptance_pdf(t, v, tau, m, s, t_min, t_max))

In [14]:
# test for unity
print("Integral of the acceptance pdf:", integrate.quad(lambda t: acceptance_pdf(t, v, tau, m, 0.03, 0, 20), 0, 20)[0])

Integral of the acceptance pdf: 0.9999999999999997


In [15]:
# generate data with time acceptance
## modify box method the use acceptance pdf
def acceptance_box_method(tau, v, m, s, runs, t_min, t_max):
    f_max = 5
    # using the box method to generate values from the pdf
    ## generate random numbers uniformly [t_min, t_max]
    ts = []
    while len(ts) < runs:
        x = np.random.uniform(t_min, t_max, runs)
        ## compute the pdf for these
        y1_s = acceptance_pdf(x, v, tau, m, s, t_min, t_max)
        ## generate a second set of random numbers uniformly [0, f_max]
        y2_s = np.random.uniform(0, f_max, runs)
        x = x[y2_s < y1_s]
        ts.extend(x)
    if len(ts) > runs:
        return ts[:runs]
    return ts

data = acceptance_box_method(tau, v, m, 0.03, 10_000, 0, 20)
# modify parameter fitting to include acceptance
def acceptance_parameter_fitting(data):
    binned_data, bins = np.histogram(data, bins=100, density=True)
    cost = BinnedNLL(binned_data, bins, acceptance_cdf)
    minuit = Minuit(cost, tau=1.5, v=0.1, m=20, s=0.03, t_min=t_min, t_max=t_max)
    minuit.errordef=0.5
    minuit.fixed['t_min'] = True
    minuit.fixed['t_max'] = True
    minuit.migrad()
    minuit.hesse()
    return np.array(minuit.values)[:3], np.array(minuit.errors)[:3]

def parameter_fitting(data):
    binned_data, bins = np.histogram(data, bins=100, density=True)
    cost = BinnedNLL(binned_data, bins, cdf)
    minuit = Minuit(cost, tau=1.5, v=0.1, m=20, t_min=t_min, t_max=t_max)
    minuit.errordef=0.5
    minuit.fixed['t_min'] = True
    minuit.fixed['t_max'] = True
    minuit.migrad()
    minuit.hesse()
    return minuit.parameters[:3], np.array(minuit.values)[:3], np.array(minuit.errors)[:3]

acceptance_vals, _ = acceptance_parameter_fitting(data)
params, vals, error = parameter_fitting(data)
# calculate the shift as systematic error
shift = np.abs(acceptance_vals - vals)
for p, val, e, e_s in zip(params, vals, error, shift):
    val = np.format_float_positional(val, precision=2, unique=False, fractional=False, trim='k')
    e = np.format_float_positional(e, precision=2, unique=False, fractional=False, trim='k')
    es = np.format_float_positional(e_s, precision=2, unique=False, fractional=False, trim='k')
    print(f"{p} = {val} +/- {e} +/- {es}")

v = -0.16 +/- 0.41 +/- 0.0055
tau = 1.7 +/- 0.5 +/- 0.38
m = 20. +/- 1.7 +/- 0.00064
